# Import

In [1]:
%load_ext autoreload

In [3]:
%autoreload 2
import os
from dotenv import load_dotenv
import sys
import os
import networkx as nx
import ipywidgets as widgets
from collections import defaultdict
from tqdm import tqdm

# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Load environment variables from .env file
load_dotenv()

from src.dialogue_generation import get_vocab_from_dialogue, update_vocab_usage
from src.dialogue_generation import generate_story_plan, generate_dialogue_prompt, generate_dialogue, generate_recap
from src.audio_generation import  text_to_speech, play_audio, generate_audio_from_dialogue, generate_normal_and_fast_audio, generate_translated_phrase_audio, join_audio_segments, export_audio
from src.phrase import correct_phrases, generate_practice_phrases_from_dialogue
from src.initialise import initialise_usage_data
from src.utils import save_json, convert_defaultdict, save_defaultdict, load_json, create_pdf_booklet
from src.translation import translate_dialogue, translate_phrases

STORY_DATA_PATH = "../outputs/story_data.json"


## Setup Google Cloud credentials and prerequisites
You will need a Google Project with the following APIs enabled:
* Text to Speech
* Translate
* Vertex AI with the following Anthropic models enabled (from the model garden)
    * Sonnet 3.5
    * Haiku
* Add your GOOGLE_PROJECT_ID to the .env file

You should alter src/config.json which contains your target language.


In [4]:
from google.auth import default
credentials, project = default()

# Audio Language Trainer Workflow

The aim of this project is to create audio material for you to practise a foreign language. It needs to be engaging and be tailored to words you want to practise. 

The overall steps we follow are:

1. Create an outline story plan based on a theme you select (e.g. 'an adventure', 'a holiday in Rome'). An LLM produces a story plan following a typical story arc (exposition, rising action, climax, falling action, resolution). This ensures an engaging plot.
2. Flesh out the story using your practice vocabulary and grammatical concepts. Vocab and concepts are sampled from lists you provide in the 'data' folder (vocab_usage.json and grammar_concepts_usage.json), with sampling being skewed towards words you haven't heard yet. The output here is a dialogue between two people (Sam and Alex).

Recaps are generated between each story part so when the LLM generates the next dialogue it logically continues from the previous one.

3. The dialogue is broken up into shorter practice phrases via a 'language graph' concept, so we give you not just the long-form dialogue to listen and practise to, but smaller, mixed-up phrases based on the vocab in the story, starting small and buliding to more complex phrases.
4. Your vocab list is updated based on the produced dialogue.
5. The smaller phrases and main dialogue are translated into your target language and convert to speech.
6. Research shows that listening to double-speed audio (on words you already known) can help with your listening comprehension for a foreign language (it helps the brain with the ability to separate distinct words). We therefore create a fast version of the dialogue for listening practice.
7. The audio files are stiched together to create an MP3 file for each part in the story (there are 5 parts to the story). The stages for each audio lesson are: 
* dialogue in the target language
* practice phrases of the form 'how do you say: "practice phrase' in 'target language'?". A pause (where you speak in the foreign language), then the correct translation is played twice, first fast, then slow.
* repeat of the dialogue in the target language so you can satisfy yourself you understand it properly
* 12 repeated playings of the fast version of the dialogue to improve your listening comprehension.

The intent is then you would listen to the next audio lesson in the story.


## Setup your vocab and grammatical concepts
You should populate or edit
* known_vocab_list.json 
* grammar_concepts.json

### Initiliase the vocab and grammar counters
This creates vocab_usage.json (setting all values to 0) and grammar_concepts_usage.json (setting all values to 'true' and counts to 0)

You can tweak these to minimise what words and concepts you are exposed to

In [4]:
initialise_usage_data(overwrite=False) #the overwrite commands stops you wiping all your usage data if it already exists

Data saved to ../data/grammar_concepts_usage.json
Data saved to ../data/vocab_usage.json


# Being Lesson Generation

## Create a story plan

In [18]:
story_plan = generate_story_plan(story_guide = "a mystery", test = False) #the test parameter will provide pre-canned responses avoiding LLM costs
story_plan

Data saved to ../outputs/story_plan.json


{'exposition': 'Alex and Sam are exchange students sharing an apartment in a foreign city. They notice strange noises coming from the apartment next door late at night.',
 'rising_action': "The two friends decide to investigate the mysterious sounds. They start asking neighbors for information and observing the apartment's occupants.",
 'climax': 'One night, Alex and Sam follow their neighbor to an abandoned building where they witness a secret meeting.',
 'falling_action': 'Confused and worried, they debate whether to inform the authorities or confront their neighbor directly. They gather more evidence about the secret activities.',
 'resolution': 'Alex and Sam discover their neighbor is organizing surprise language exchange events for international students. They join the group and make new friends from around the world.'}

## Create all dialogue

1. Create dialouge LLM prompt based on the story part
2. LLM generates dialogue
3. LLM generates recap
4. move to next story part and repeat


In [20]:
PAY_FOR_LLM = True

if PAY_FOR_LLM:
    story_data_dict = defaultdict(lambda: defaultdict(str))
    recap = "This is the beginning of the story."
    for step, story_part in enumerate(list(story_plan.keys())):
        prompt = generate_dialogue_prompt(story_part=story_part,
                                        story_part_outline=story_plan[story_part],
                                        last_recap = recap,
                                        verb_count=10,
                                        verb_use_count=5,
                                        vocab_count=30,
                                        vocab_use_count=10,
                                        grammar_concept_count=5,
                                        grammar_use_count=3)
        dialogue = generate_dialogue(prompt)
        recap = generate_recap(dialogue, test=False)
        story_data_dict[story_part]["dialogue_generation_prompt"] = prompt
        story_data_dict[story_part]["dialogue"] = dialogue
        story_data_dict[story_part]["recap"] = recap


In [21]:
save_defaultdict(story_data_dict, STORY_DATA_PATH)


Data saved to ../outputs/story_data.json


### Update the vocab lists based on the dialogue

The grammatical concepts are updated during prompt creation as it is more difficult to extract these from the dialogue

In [22]:
for story_part in story_plan.keys():
    dialogue = story_data_dict[story_part]["dialogue"]
    vocab_used = get_vocab_from_dialogue(dialogue)
    update_vocab_usage(vocab_used)

Data saved to ../data/vocab_usage.json
Data saved to ../data/vocab_usage.json
Data saved to ../data/vocab_usage.json
Data saved to ../data/vocab_usage.json
Data saved to ../data/vocab_usage.json


### Build phrases from dialogue

Here we:
1. Break up the dialogue into separate sentences. For this bit we don't care who the speaker is, we just want to create different phrases of different lengths and combinations based on the vocab int the dialogue
2. We use another LLM call to do this, with some one-shot learning

In [17]:
story_data_dict = load_json(STORY_DATA_PATH)

Searching for config.json...
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\notebooks\config.json
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Found config file at: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Successfully loaded config from: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Multiple country codes available for en: en-AU, en-GB, en-IN, en-US
Config loader initialized.
Config file location: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Current working directory: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\notebooks
Searching for config.json...
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\notebooks\config.json
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Found config file at: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\con

In [9]:
for story_part in story_data_dict:
    dialogue = story_data_dict[story_part]["dialogue"]
    story_data_dict[story_part]["corrected_phrase_list"] = generate_practice_phrases_from_dialogue(dialogue)

In [10]:
save_defaultdict(story_data_dict, STORY_DATA_PATH)

Data saved to ../data/story_data.json


### Translate dialogue and phrases

In [26]:
PAY_FOR_TRANSLATE_API = True

if PAY_FOR_TRANSLATE_API:

    for story_part in tqdm(story_data_dict):
        dialogue = story_data_dict[story_part]["dialogue"]
        translated_dialogue = translate_dialogue(dialogue)

        corrected_phrase_list = story_data_dict[story_part]["corrected_phrase_list"]
        translated_phrase_list = translate_phrases(corrected_phrase_list)

        story_data_dict[story_part]["translated_dialogue"] = translated_dialogue
        story_data_dict[story_part]["translated_phrase_list"] = translated_phrase_list


 40%|████      | 2/5 [00:08<00:12,  4.09s/it]


BadRequest: 400 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: Required Text [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'q', 'description': 'No text queries provided.'}]}]

In [12]:
save_defaultdict(story_data_dict, STORY_DATA_PATH)

Data saved to ../data/story_data.json


## Generate Audio Lesson

The steps here are
1. The target language dialogue at normal speed
2. Each corrected and translated phrase in the form english - target fast - target slow
3. Each dialogue utterance in the form english - target fast - target slow
4. The 2 x sped up target language dialogue

In [31]:
%autoreload 2
from src.audio_generation import generate_audio_from_dialogue
test_audio = generate_audio_from_dialogue(story_data_dict["exposition"]["translated_dialogue"][0:2])

100%|██████████| 2/2 [00:03<00:00,  1.82s/it]


In [34]:
test_audio

In [35]:
PAY_FOR_TEXT_TO_SPEECH = True

audio_lessons = []

if PAY_FOR_TEXT_TO_SPEECH:
    for story_part in tqdm(story_data_dict):

        single_audio_lesson = []
        single_audio_lesson_filename = story_part + ".mp3"
        translated_dialogue_audio_segments = generate_audio_from_dialogue(story_data_dict[story_part]["translated_dialogue"])
        normal_translated_clip, fast_translated_clips = generate_normal_and_fast_audio(translated_dialogue_audio_segments)
        single_audio_lesson.append(normal_translated_clip)

        #now do phrases
        for translated_phrase in tqdm(story_data_dict[story_part]["translated_phrase_list"]):
            phrase_audio = generate_translated_phrase_audio(translated_phrase)
            single_audio_lesson.append(phrase_audio)

        #now add fast bit at the end
        single_audio_lesson.append(fast_translated_clips)
        single_audio_lesson = join_audio_segments(single_audio_lesson)
        audio_lessons.append(single_audio_lesson)
        
        export_audio(single_audio_lesson, f"../outputs/{single_audio_lesson_filename}")
    
    full_lesson = join_audio_segments(audio_lessons, gap_ms=3000)
    export_audio(full_lesson, filename="../outputs/full_lesson.mp3")


100%|██████████| 5/5 [20:27<00:00, 245.56s/it]


## Create PDF Booklet
So you can see the spelling of the phrases and dialogue

In [38]:
story_data_dict["exposition"]["translated_dialogue"]

[{'speaker': 'Alex', 'text': 'Добро јутро, Сам! Да ли наручујете пециво?'},
 {'speaker': 'Sam', 'text': 'Да, наручујем мафин са циметом. Што то једеш?'},
 {'speaker': 'Alex',
  'text': 'Раније сам наручио кроасан. Било је скупо, али тако добро!'},
 {'speaker': 'Sam',
  'text': 'Свиђа ми се специјални латте овде. Да ли вам треба бадемово млеко?'},
 {'speaker': 'Alex',
  'text': 'Не хвала. Пијем без кофеина са овсеним млеком. Није горко.'},
 {'speaker': 'Sam', 'text': 'Велики! Да ли преузимате своју наруџбу?'},
 {'speaker': 'Alex',
  'text': 'Не, ја једем овде. Можете ли ми донети салвету, молим вас?'},
 {'speaker': 'Sam', 'text': 'Наравно! Донећу ти салвету и кашику.'}]

In [18]:

create_pdf_booklet(story_data_dict, output_filename="../outputs/story_booklet.pdf")
